In [1]:
options(bitmapType="cairo")
library(limma)
library(skitools)
library(skidb)
library(stringr)
library(data.table)
library(splines)

Loading required package: GenomicRanges
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit

Loading required package: S4Vectors
Loading required pac

In [2]:
# Reading files in. This is expression data
expr = fread("/gpfs/commons/home/deshpandea-934/data/Histomics_LUAD/csv_featurs/output/Filtered_LUAD_exp.csv", header = T)

Read 20531 rows and 394 (of 394) columns from 0.053 GB file in 00:00:04


In [3]:
# This file has the proportions of cells belongigng to 14 clusters obtained by applying K means to all 406
# data points
prop = fread("/gpfs/commons/home/deshpandea-934/data/Histomics_LUAD/csv_featurs/output/All_samples_prop.csv", header = T)

In [4]:
# Converting gene names to index/rownames
expr = as.data.frame(expr)
rownames(expr) <- expr$gene_id
head(rownames(expr))
expr$gene_id <- NULL

[1] "?|100130426" "?|100133144" "?|100134869" "?|10357"     "?|10431"    
[6] "?|136542"

In [5]:
# Applied filter to genes with mean >1 and number of samples showing expression > 100
expr.min = apply(expr, 1, min)
expr.max = apply(expr, 1, max)
expr.mean = apply(expr, 1, mean)
expr.var = apply(expr, 1, var)
egene = (expr.mean>1 | expr.max > 100)
sum(egene)
sum(!egene)

[1] 18217

[1] 2314

In [1]:
# Since we are dealing with continuous variable, the limma analysis is a bit different.
# More info here: https://support.bioconductor.org/p/76638/

In [47]:
cluster0 <- prop$'0' # created an object with proportions of cluster0
X <- ns(cluster0, df = 5) # fitted a spline for each gene with proportion of cells in cluster0 as covariate
design <- model.matrix(~X) # Got design matrix from this
y = voom(expr[egene, ], design, plot = FALSE) #Rest of analysis follows ususal limma steps
fit = eBayes(lmFit(y, design))
results0_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results0_1 = as.data.frame(results0_1)
cl_0 <- subset(results0_1, adj.P.Val <= 0.05)

# We use the P values from above but the logFold can be obtained by using age directly as a covariate in design

design2 <- model.matrix(~cluster0)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results0_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results0_2 = as.data.frame(results0_2)
cl_0 <- cbind(cl_0, subset(results0_2, (rownames(results0_2) %in% rownames(cl_0)), select = "logFC"))
cl_0[order(-cl_0$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
EREG|2069,-0.1027245,-0.2422926,0.4513197,2.790134,6.8849418,1.7012037,10.105746,4.082312e-09,7.436748e-05,29.46931
LEMD1|93273,-3.2878184,-3.4504841,-1.0887814,-8.218969,-0.4837581,0.0833381,7.780642,5.385434e-07,4.905322e-03,4.29760


In [2]:
# Same analyis carried for each cluster type

In [48]:
cluster1 <- prop$'1'
X <- ns(cluster1, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results1_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results1_1 = as.data.frame(results1_1)
cl_1 <- subset(results1_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster1)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results1_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results1_2 = as.data.frame(results1_2)
cl_1 <- cbind(cl_1, subset(results1_2, (rownames(results1_2) %in% rownames(cl_1)), select = "logFC"))
cl_1[order(-cl_1$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
CYP3A5|1577,-1.695991,-0.7434299,-4.407967,-2.287065,-0.5178009,0.3357686,7.218023,1.76433e-06,0.0321408,-45.12635


In [49]:
cluster2 <- prop$'2'
X <- ns(cluster2, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results2_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results2_1 = as.data.frame(results2_1)
cl_2 <- subset(results2_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster2)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results2_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results2_2 = as.data.frame(results2_2)
cl_2 <- cbind(cl_2, subset(results2_2, (rownames(results2_2) %in% rownames(cl_2)), select = "logFC"))
cl_2[order(-cl_2$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
SCN2A|6326,0.36867194,1.18490210,0.32193604,0.9331335,2.96331499,-1.6194648,4.459513,5.794492e-04,3.744289e-02,22.49700
PCDHB5|26167,0.24130368,0.37696910,1.40209216,-0.8449206,1.43087488,1.8013443,5.837219,3.242923e-05,8.687696e-03,19.26512
ADAMTS18|170692,0.04023093,0.80048484,0.95602880,0.4647380,1.01665745,-1.5534809,4.157518,1.082708e-03,4.819016e-02,18.36781
CDH2|1000,-0.46599957,0.66159326,-0.54768596,-1.5807570,2.06232045,1.0603839,4.653395,3.872406e-04,3.135272e-02,16.59820
PGA3|643834,1.60602748,3.39988328,-1.89936361,7.5219084,6.92469055,-3.3521308,9.325030,2.091696e-08,7.694354e-05,16.32156
LGR5|8549,-1.87705873,-0.26532918,-1.32032305,-3.4419335,1.34888905,-0.2999292,5.376655,8.536224e-05,1.447096e-02,15.90128
PGA4|643847,0.94304272,1.96828198,-0.93410847,4.7705849,4.83235239,-4.5928227,6.877615,3.618344e-06,2.636615e-03,15.65340
HTR2B|3357,-1.65187750,-1.61432820,-0.46125479,-3.3646874,1.65311293,0.6921282,4.316180,7.799393e-04,4.214693e-02,15.41660
CHRNA1|1134,-1.73316410,-0.39788581,-1.40175763,-2.9622130,1.68887052,-0.2916715,5.410696,7.947732e-05,1.405668e-02,14.97001
EYA4|2070,0.12089510,1.05558146,0.60440932,-0.7470283,0.60724403,-0.1164148,4.759761,3.102730e-04,2.826122e-02,14.57727


In [50]:
cluster3 <- prop$'3'
X <- ns(cluster3, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results3_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results3_1 = as.data.frame(results3_1)
cl_3 <- subset(results3_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster3)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results3_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results3_2 = as.data.frame(results3_2)
cl_3 <- cbind(cl_3, subset(results3_2, (rownames(results3_2) %in% rownames(cl_3)), select = "logFC"))
cl_3[order(-cl_3$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
EMX2|2018,0.4166283,0.6103688,-1.107007,1.926735,3.705429,-4.63204320,7.414129,1.166565e-06,0.010625654,10.09947
L1CAM|3897,0.1624180,1.3262033,-2.460470,3.814636,5.118908,-0.08060725,8.493620,1.199514e-07,0.002185155,1.89217


In [51]:
cluster4 <- prop$'4'
X <- ns(cluster4, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results4_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results4_1 = as.data.frame(results4_1)
cl_4 <- subset(results4_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster4)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results4_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results4_2 = as.data.frame(results4_2)
cl_4 <- cbind(cl_4, subset(results4_2, (rownames(results4_2) %in% rownames(cl_4)), select = "logFC"))
cl_4[order(-cl_4$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
RBMY2FP|159162,-0.03812987,0.2540424,-0.5106573,1.349798,2.4794156,-5.20102295,6.981963,2.903314e-06,1.157832e-02,8.3795491
APOH|350,-4.85522427,-6.0351643,-1.5410295,-13.092495,-1.4012151,0.43288349,7.450202,1.081092e-06,6.564750e-03,6.1413345
NRG1|3084,-3.57870879,-4.0195870,-2.0833974,-7.830395,0.9304858,-0.01256007,6.550226,7.218646e-06,1.878601e-02,2.3359120
FUNDC1|139341,-1.32094064,-1.5693404,-0.3685382,-3.568225,-0.8972275,4.02509882,6.429252,9.316541e-06,2.121493e-02,-0.8213417
ALB|213,-6.70063982,-8.2310040,-3.2174915,-17.298388,-1.2237734,-2.16552011,6.823592,4.055189e-06,1.231223e-02,-3.1993320
ZNF673|55634,-1.73196089,-1.9004311,-0.7364698,-4.578700,-1.1448732,3.41584770,12.642891,2.127761e-11,3.876142e-07,-3.2740970
ZNF674|641339,-1.80245861,-1.8511073,-1.1851171,-4.767457,-1.1265011,1.69830430,11.216580,4.042741e-10,3.682331e-06,-7.6750847
GSTA2|2939,-4.39050675,-6.1443272,-2.3394605,-14.449989,-2.2274532,-0.36933364,6.939135,3.177888e-06,1.157832e-02,-13.0436422


In [52]:
cluster5 <- prop$'5'
X <- ns(cluster5, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results5_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results5_1 = as.data.frame(results5_1)
cl_5 <- subset(results5_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster5)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results5_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results5_2 = as.data.frame(results5_2)
cl_5 <- cbind(cl_5, subset(results5_2, (rownames(results5_2) %in% rownames(cl_5)), select = "logFC"))
cl_5[order(-cl_5$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
SERPINB5|5268,2.02105742,2.597873,1.0813562,5.319911,5.825244,0.6973852,6.543397,7.323222e-06,8.443490e-04,128.55226
COL11A1|1301,4.04004336,3.718154,4.6048584,6.834223,2.742892,3.8420476,6.693499,5.335763e-06,6.523596e-04,122.34604
APCDD1L|164284,2.10610004,2.806587,0.6282238,4.502506,5.824439,-0.7187729,13.182933,7.032629e-12,1.281134e-07,119.25496
GREM1|26585,2.82899726,3.202886,3.2052738,6.565775,2.679438,4.8037098,7.307426,1.461029e-06,2.694075e-04,108.74738
PPAPDC1A|196051,2.62175207,2.233226,3.0288012,4.562721,2.678396,-0.2716631,7.526850,9.196812e-07,2.043150e-04,107.14900
HOXB9|3219,2.06973405,2.062875,1.3591628,1.057210,3.279377,-0.7780288,4.332926,7.533520e-04,1.982156e-02,102.47513
RAB3B|5865,1.66372295,2.460357,0.6647241,4.267288,4.368215,-2.2198381,5.317694,9.659902e-05,4.888179e-03,96.26119
SPOCK1|6695,2.31320584,3.045782,1.7489391,5.963677,3.754179,2.3150683,7.708542,6.269458e-07,1.571028e-04,94.93792
TNNT1|7138,3.97842619,5.762837,2.2591329,11.328450,4.323129,2.1899124,4.420774,6.279440e-04,1.762597e-02,93.27745
CCL26|10344,2.38179237,2.475324,2.2298260,5.078274,2.797636,-2.0066310,6.793753,4.318596e-06,5.540271e-04,91.75941


In [53]:
cluster6 <- prop$'6'
X <- ns(cluster6, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results6_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results6_1 = as.data.frame(results6_1)
cl_6 <- subset(results6_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster6)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results6_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results6_2 = as.data.frame(results6_2)
cl_6 <- cbind(cl_6, subset(results6_2, (rownames(results6_2) %in% rownames(cl_6)), select = "logFC"))
cl_6[order(-cl_6$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
ADAM6|8755,1.61531760,2.755976189,1.81213615,3.8386282,1.5128045,11.83943090,6.318580,1.176469e-05,0.0055721829,25.34974
CD19|930,0.35319440,1.538133094,0.85303891,1.7734238,2.3336379,0.93075886,4.907817,2.278035e-04,0.0292576604,25.20899
KIAA0125|9834,1.22340982,2.136506672,1.77616283,2.9651114,1.6527901,1.49876253,5.297791,1.007154e-04,0.0174886535,24.61056
C8orf80|389643,0.58387753,1.545776640,1.10624164,1.8545851,2.0878814,0.24376511,7.891087,4.266696e-07,0.0007772640,24.42925
AMPD1|270,1.69419438,2.891834301,1.51018507,3.6867984,1.7873320,-1.23539793,5.555001,5.870026e-05,0.0132223060,24.01241
CD79A|973,0.21237157,1.520577797,0.98966627,1.3608566,1.3230542,4.23276210,5.525671,6.243064e-05,0.0132223060,23.64536
LOC96610|96610,0.94410268,2.295977934,1.55336486,3.0839991,0.9259997,10.63481615,6.116237,1.802097e-05,0.0065657600,23.30985
TNFRSF17|608,1.16082228,2.315807139,1.72622355,3.2627033,1.2522014,1.16549113,5.297382,1.008019e-04,0.0174886535,23.16255
FCRL5|83416,1.38986747,2.251699915,1.65617539,3.1792509,1.7500252,2.88208243,4.519033,5.120729e-04,0.0447505370,22.63092
FCRL3|115352,-0.04509838,0.404675821,0.81521282,-0.1687313,1.7828868,0.30686158,4.856458,2.535904e-04,0.0313362086,22.10198


In [54]:
cluster7 <- prop$'7'
X <- ns(cluster7, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results7_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results7_1 = as.data.frame(results7_1)
cl_7 <- subset(results7_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster7)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results7_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results7_2 = as.data.frame(results7_2)
cl_7 <- cbind(cl_7, subset(results7_2, (rownames(results7_2) %in% rownames(cl_7)), select = "logFC"))
cl_7[order(-cl_7$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
CCDC36|339834,2.755829381,1.8334829,2.1077866,5.268717,0.4382708,-1.859654,7.799058,5.180221e-07,0.009436808,4.487665
C10orf116|10974,-1.921367759,-3.1763513,0.7995581,-4.890210,-2.4191230,3.915421,6.456352,8.798895e-06,0.046217512,-2.143581
CGB1|114335,-2.597722375,-3.0836804,-1.2834754,-7.111845,-1.1800330,-4.940777,6.651213,5.833603e-06,0.046217512,-5.205375
UBE2E2|7325,-0.007851517,-0.5521864,-0.4674985,-1.755322,-0.3556329,3.950972,6.388690,1.014822e-05,0.046217512,-6.011211


In [55]:
cluster8 <- prop$'8'
X <- ns(cluster8, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results8_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results8_1 = as.data.frame(results8_1)
cl_8 <- subset(results8_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster8)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results8_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results8_2 = as.data.frame(results8_2)
cl_8 <- cbind(cl_8, subset(results8_2, (rownames(results8_2) %in% rownames(cl_8)), select = "logFC"))
cl_8[order(-cl_8$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
SPRR1B|6699,0.14847960,0.996475046,-0.29824042,6.5926188,8.211577,-1.10626895,7.900208,4.185582e-07,5.446339e-04,73.15130704
CSF2|1437,0.80070292,-0.131648482,-0.01264273,1.8380374,6.657076,-0.39064535,19.015324,6.101245e-17,1.852440e-13,70.25683669
SPRR2F|6705,-0.28455423,0.613855906,-0.72643153,5.8605709,8.988044,-3.84333630,19.080878,5.369425e-17,1.852440e-13,69.37878938
MAGEB1|4112,0.04363956,0.049402587,-0.61513462,3.3480129,7.813496,-4.70669042,18.251763,2.716936e-16,7.070632e-13,67.59419518
VCX3A|51481,1.41196849,1.816756308,-1.07273756,7.1080765,10.147624,-3.43279208,30.874201,1.760230e-26,8.016528e-23,67.38143399
CSMD1|64478,1.89507363,2.035651757,0.71697775,7.2460557,6.394213,-2.02769358,5.857715,3.106004e-05,2.052960e-02,65.54214562
FAM9C|171484,0.51580335,0.241472373,-0.07433176,3.7418498,7.237297,-4.58573715,14.279762,7.529291e-13,1.371611e-09,65.15344466
SPRR2D|6703,-1.37938371,-0.463032706,-0.98359519,1.9799182,6.342846,-2.00624031,5.317971,9.654372e-05,4.521491e-02,62.15160104
SPANXB2|100133171,0.19467025,0.668684881,-1.21174530,5.5905944,9.990385,-4.69227877,34.931843,1.580970e-29,1.440026e-25,60.84443132
ADCY8|114,-0.25709401,-0.005917705,-0.50321972,3.1020343,6.453069,-3.95777314,6.505092,7.939493e-06,6.574261e-03,53.43605786


In [56]:
cluster9 <- prop$'9'
X <- ns(cluster9, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results9_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results9_1 = as.data.frame(results9_1)
cl_9 <- subset(results9_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster9)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results9_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results9_2 = as.data.frame(results9_2)
cl_9 <- cbind(cl_9, subset(results9_2, (rownames(results9_2) %in% rownames(cl_9)), select = "logFC"))
cl_9[order(-cl_9$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
C21orf99|149992,0.20214596,0.5212936,0.830395971,3.2532593,2.52562644,-4.53921195,4.487833,5.463588e-04,0.022986185,14.857698
PLA2G2A|5320,-1.95561283,-2.6821929,-0.662170246,-1.7084780,2.71009481,0.45758746,4.602090,4.308687e-04,0.019760765,8.728368
LILRA2|11027,2.39166653,1.2091863,2.852898587,4.7905937,0.61754088,1.26639221,4.061311,1.320267e-03,0.039623250,6.981940
GATC|283459,-0.80357561,-0.6857485,0.554528173,-1.0448739,-0.51035675,0.81677962,4.432494,6.128611e-04,0.024509485,6.882022
DNASE1L2|1775,-0.87845383,-1.7199657,1.906925511,-3.6427775,-2.94457179,-1.57156637,4.114242,1.183813e-03,0.037117945,6.460675
SIGLEC5|8778,-0.01082726,-1.0524960,0.375303467,-0.2632934,1.80295031,1.21681946,4.542899,4.872991e-04,0.021390669,5.762211
RPS3|6188,-1.07363668,-0.6771244,0.118982904,-1.2338896,-0.49979239,10.11492896,5.300175,1.002139e-04,0.007431107,5.755980
MTFR1|9650,-0.05812105,0.3970521,0.743796143,0.5310480,-0.71571185,5.10977113,4.367707,7.009783e-04,0.026827145,5.632562
NLE1|54475,-0.48800201,-0.4917854,0.502663692,-0.7429225,-0.28880324,4.05001412,4.142690,1.116344e-03,0.035866729,5.576760
RAB5C|5878,0.17829483,0.1368573,0.608399455,0.8151464,0.41984161,7.44784716,4.859264,2.521106e-04,0.013587866,5.424052


In [57]:
cluster10 <- prop$'10'
X <- ns(cluster10, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results10_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results10_1 = as.data.frame(results10_1)
cl_10 <- subset(results10_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster10)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results10_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results10_2 = as.data.frame(results10_2)
cl_10 <- cbind(cl_10, subset(results10_2, (rownames(results10_2) %in% rownames(cl_10)), select = "logFC"))
cl_10[order(-cl_10$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
TNNC1|7134,1.4031091,2.1598527,1.1200773,3.1542582,2.1997584,1.6022381,5.472430,6.981674e-05,0.0493387071,56.154465
LHCGR|3973,-0.1492568,0.8481916,-1.0536865,1.9997480,3.8445522,-3.6555714,6.144111,1.699333e-05,0.0238128831,50.574184
PI16|221476,-1.0932566,0.6622814,-1.1771983,1.2105643,2.9762561,-1.3997659,5.468345,7.041809e-05,0.0493387071,46.833578
SULT1A2|6799,0.7872918,0.9259712,1.3256494,1.9508520,1.2459025,1.7609322,5.598572,5.356549e-05,0.0464667831,45.531253
ICAM5|7087,-0.1023897,0.6685015,-0.9569515,1.1461297,3.0296155,2.2349813,6.868735,3.686711e-06,0.0095944008,41.418913
ABCA17P|650655,-1.0320126,-0.7676417,-1.0835557,-0.2601006,3.1116906,-1.4767273,6.519019,7.709640e-06,0.0156051672,40.011831
MESP1|55897,0.9958418,0.9312586,0.2401198,1.7251355,2.3911907,1.4918980,5.897333,2.857504e-05,0.0325344706,39.029815
CCDC78|124093,-0.6753375,-0.8256994,-0.6505017,-0.6548707,2.5195649,1.5120046,5.569097,5.698788e-05,0.0471885576,37.572456
PLIN1|5346,0.1626049,0.7753861,-0.4631425,1.7311244,2.5580525,-2.0247627,5.624358,5.073994e-05,0.0464667831,36.747586
TMEM88|92162,0.4882438,1.1565176,0.2018513,2.4575448,1.8228856,0.5631977,7.981749,3.524668e-07,0.0024103144,36.353909


In [59]:
cluster11 <- prop$'11'
X <- ns(cluster11, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results11_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results11_1 = as.data.frame(results11_1)
cl_11 <- subset(results11_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster11)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results11_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results11_2 = as.data.frame(results11_2)
cl_11 <- cbind(cl_11, subset(results11_2, (rownames(results11_2) %in% rownames(cl_11)), select = "logFC"))
cl_11[order(-cl_11$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
CNNM4|26504,-0.37202009,0.08956224,0.3651719,0.2338551,0.4192526,5.46407488,6.199422,1.512382e-05,0.0344388269,10.921563
CLDN22|53842,0.01644795,0.24160416,-0.2551525,0.9539117,1.7666555,-5.24335725,7.519416,9.342214e-07,0.0056729039,9.508551
TEX13B|56156,0.09169831,0.40565061,-0.2232217,1.4539060,1.8268909,-5.10658195,9.425653,1.693837e-08,0.0003085663,9.498374
SYT12|91683,-2.92865164,-0.48128904,-2.3621705,-1.0867382,0.8612226,3.29462942,6.761427,4.623415e-06,0.0120321077,8.693965
SPATS2L|26010,-0.78793320,0.28374020,-0.5225439,0.3780364,0.5979605,7.00519512,7.258050,1.621443e-06,0.0059075661,7.268503
NEDD4|4734,-0.81168941,0.17707918,-0.9347734,-0.7701455,0.6772519,3.79526308,7.343688,1.353435e-06,0.0059075661,6.031087
ARHGAP10|79658,-1.08072250,-0.02509279,-0.4321466,-0.5778198,-0.6117457,3.72186042,7.091632,2.303520e-06,0.0069938711,2.186076
IVL|3713,-5.78376335,-3.40055139,-3.4146584,-8.5040736,-3.1241618,-0.02942383,7.532315,9.091472e-07,0.0056729039,-11.425943


In [60]:
cluster12 <- prop$'12'
X <- ns(cluster12, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results12_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results12_1 = as.data.frame(results12_1)
cl_12 <- subset(results12_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster12)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results12_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results12_2 = as.data.frame(results12_2)
cl_12 <- cbind(cl_12, subset(results12_2, (rownames(results12_2) %in% rownames(cl_12)), select = "logFC"))
cl_12[order(-cl_12$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
SPAG11A|653423,-1.972197,-2.532165,-0.7753443,-5.28834,-0.8931081,-5.153954,9.068701,3.581503e-08,0.0006524424,-6.739489


In [61]:
cluster13 <- prop$'13'
X <- ns(cluster13, df = 5)
design <- model.matrix(~X)
y = voom(expr[egene, ], design, plot = FALSE)
fit = eBayes(lmFit(y, design))
results13_1 <- topTable(fit, coef=2:ncol(design), n=Inf, sort.by="none")
results13_1 = as.data.frame(results13_1)
cl_13 <- subset(results13_1, adj.P.Val <= 0.05)


design2 <- model.matrix(~cluster13)
y2 = voom(expr[egene, ], design2, plot = FALSE)
fit2 = eBayes(lmFit(y2, design2))
results13_2 <- topTable(fit2, coef=2, n=Inf, sort.by="none")
results13_2 = as.data.frame(results13_2)
cl_13 <- cbind(cl_13, subset(results13_2, (rownames(results13_2) %in% rownames(cl_13)), select = "logFC"))
cl_13[order(-cl_13$logFC),]

,X1,X2,X3,X4,X5,AveExpr,F,P.Value,adj.P.Val,logFC
GKN2|200504,2.18008657,1.73672356,1.06106501,7.5178952,6.2629111,0.09429372,6.977879,2.928436e-06,0.001766696,29.653174
NR1H4|9971,0.49093750,0.03178429,1.48254535,1.8577272,2.3831304,-3.60668527,3.648008,3.079838e-03,0.044422338,26.790097
TFF2|7032,-0.15439730,0.81236234,-0.78416530,3.4664883,6.8372601,-3.41881680,3.679920,2.885824e-03,0.042985322,25.153751
ABO|28,1.81841077,2.01272717,1.15129496,4.4225747,2.6951562,1.88733816,4.912889,2.254065e-04,0.011121738,24.124999
IYD|389434,2.26276917,3.59487701,0.26849930,8.0761728,4.9869918,1.92471707,6.989833,2.855498e-06,0.001766696,23.826334
FAM177B|400823,0.38946325,1.46365951,0.06487400,4.5961446,4.5348830,-0.59935626,4.524082,5.067354e-04,0.016878985,22.675381
POF1B|79983,1.61310755,2.09428181,1.17821581,7.0565504,3.5269451,2.42366611,4.522797,5.080909e-04,0.016878985,22.667860
DPCR1|135656,1.71626462,1.71078086,0.08848643,4.1071942,4.7730287,1.42239930,4.165258,1.065546e-03,0.024822313,21.249604
SPINK4|27290,1.35388702,1.54356407,-0.13087347,6.2147825,6.3885388,-3.66855956,5.444925,7.396758e-05,0.006462740,18.984815
SHH|6469,1.18858526,1.35947485,0.43495426,4.1671235,3.6076995,-0.73444812,3.681175,2.878445e-03,0.042945641,18.143572
